### Import all data sets

In [ ]:
import pandas as pd
import numpy as np

stnl1 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/ogData/stnl_activity_subset_1.csv")
stnl2 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/ogData/stnl_activity_subset_2.csv")
stnl3 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/ogData/stnl_activity_subset_3.csv")
stnl4 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/ogData/stnl_activity_subset_4.csv")

stnl = pd.concat([stnl1, stnl2, stnl3, stnl4], ignore_index=True)
bp = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/ogData/bp_activity_subset.csv")
sth = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/ogData/new_sth_subset.csv")
lost1 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/lostData/lost_stnl_and_bp_activity_subset_1.csv")
lost2 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/lostData/lost_stnl_and_bp_activity_subset_2.csv")
lost3 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/lostData/lost_stnl_and_bp_activity_subset_3.csv")
lost4 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/lostData/lost_stnl_and_bp_activity_subset_4.csv")
lost5 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/lostData/lost_stnl_and_bp_activity_subset_5.csv")

# Clean process
- Sort by date occur
- Run through new function *(filter_sth)*
- Run through cleaning process *(transform_data_AC)* to find 

### Sort Data

In [19]:
stnl['Date'] = pd.to_datetime(stnl['ACTIVITY_DATETIME'])
stnl_date_sorted = stnl.sort_values(by=['CUSTOMER_KEY','Date'])

bp['Date'] = pd.to_datetime(bp['ACTIVITY_DATETIME'])
bp_date_sorted = bp.sort_values(by=['CUSTOMER_KEY','Date'])

sth['Date'] = pd.to_datetime(sth['ACTIVITY_DATETIME'])
sth_date_sorted = sth.sort_values(by=['CUSTOMER_KEY','Date'])

lost1['Date'] = pd.to_datetime(lost1['ACTIVITY_DATETIME'])
lost1_date_sorted = lost1.sort_values(by=['CUSTOMER_KEY','Date'])

lost2['Date'] = pd.to_datetime(lost2['ACTIVITY_DATETIME'])
lost2_date_sorted = lost2.sort_values(by=['CUSTOMER_KEY','Date'])

lost3['Date'] = pd.to_datetime(lost3['ACTIVITY_DATETIME'])
lost3_date_sorted = lost3.sort_values(by=['CUSTOMER_KEY','Date'])

lost4['Date'] = pd.to_datetime(lost4['ACTIVITY_DATETIME'])
lost4_date_sorted = lost4.sort_values(by=['CUSTOMER_KEY','Date'])

lost5['Date'] = pd.to_datetime(lost5['ACTIVITY_DATETIME'])
lost5_date_sorted = lost5.sort_values(by=['CUSTOMER_KEY','Date'])

### Function for creating multiple lists

In [5]:
def filter_sth(df):
    # Convert 'Date' to datetime and extract year and month
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.dropna(subset=['Date'])
    df['year'] = df['Date'].dt.year.astype(int)
    df['month'] = df['Date'].dt.month.astype(int)

    # Initialize lists to store data for years with and without "primary_purchase_sth"
    sth_list = []
    non_sth_list = []

    # Iterate through each unique customer
    for customer_id in df['CUSTOMER_KEY'].unique():
        customer_data = df[df['CUSTOMER_KEY'] == customer_id]
        years = customer_data['year'].unique()
        
        # Iterate through each year for the current customer
        for year in years:
        
            # Define the date range for June 1 to May 30 of the following year
            start_date = pd.Timestamp(year=year, month=6, day=1)
            end_date = pd.Timestamp(year=year + 1, month=5, day=30)
            
            # Filter data within the specified date range
            year_span_data = customer_data[(customer_data['Date'] >= start_date) & (customer_data['Date'] <= end_date)]
            
            # Check for "primary_purchase_sth" in activity codes
            if (year_span_data['ACTIVITY_CODE'] == "primary_purchase_sth").any():
                sth_list.append(year_span_data)
            else:
                non_sth_list.append(year_span_data)

    # Concatenate the results for years with and without the purchase activity
    sth_df = pd.concat(sth_list, ignore_index=True) if sth_list else pd.DataFrame()
    non_sth_df = pd.concat(non_sth_list, ignore_index=True) if non_sth_list else pd.DataFrame()

    return sth_df, non_sth_df

# Usage:
#sth_df, non_sth_df = filter_purchase_activity(stnl_filtered)

In [ ]:
stnl_with_sth, stnl_without_sth = filter_sth(stnl_date_sorted)

bp_with_sth, bp_without_sth = filter_sth(bp_date_sorted)

lost1_with_sth, lost1_without_sth = filter_sth(lost1_date_sorted)

lost2_with_sth, lost2_without_sth = filter_sth(lost2_date_sorted)

lost3_with_sth, lost3_without_sth = filter_sth(lost3_date_sorted)

lost4_with_sth, lost4_without_sth = filter_sth(lost4_date_sorted)

lost5_with_sth, lost5_without_sth = filter_sth(lost5_date_sorted)

### Sort for sth 
- Take out all activity after sth purchase

In [21]:
first_purchase_time = sth[sth['ACTIVITY_CODE'] == 'primary_purchase_sth'].groupby('CUSTOMER_KEY')['ACTIVITY_DATETIME'].min().reset_index()

# Step 2: Merge to get the first purchase timestamp for each customer
df = sth.merge(first_purchase_time, on='CUSTOMER_KEY', how='left', suffixes=('', '_first_purchase'))

# Step 3: Split into before and after first purchase
sth_without_sth= df[df['ACTIVITY_DATETIME'] < df['ACTIVITY_DATETIME_first_purchase']].drop(columns=['ACTIVITY_DATETIME_first_purchase'])
sth_with_sth = df[df['ACTIVITY_DATETIME'] >= df['ACTIVITY_DATETIME_first_purchase']].drop(columns=['ACTIVITY_DATETIME_first_purchase'])

### Function for cleaning process:
Gets count of each activity code for each customer in a data frame for selected activities
- LOB Seahawks 
- LOB fgi
- email_click
- email unsub
- form submission
- forward received/send
- merch purchase
- page_view
- primary purchase club/ga/single/sth/suite
- secondary purchase/sale
- survey_submission

In [7]:
def transform_data_AC(df):
    # Create a copy to manipulate
    df_copy = df.copy()

    # Make dummy variables for LOB only counting non NaNs
    df_copy['LOB_SEAHAWKS'] = df_copy['LINE_OF_BUSINESS'] == "seahawks"
    df_copy['LOB_FGI'] = df_copy['LINE_OF_BUSINESS'] == "fgi"
    
    # Drop unneeded columns
    df_dropped = df_copy.drop(columns=['CUSTOMER_ACTIVITY_KEY', 'ACTIVITY_SOURCE', 'ACTIVITY_TYPE', 'ACTIVITY_DATETIME', 
                                       'LINE_OF_BUSINESS','IS_ENGAGEMENT_ACTIVITY','Date'])

    # Store Campaign Lost if applicable
    lost = False
    if "CAMPAIGN_LOST" in df_dropped.columns:
        lost = True
        lost_column = df_dropped[["CUSTOMER_KEY", "CAMPAIGN_LOST"]].drop_duplicates(subset=["CUSTOMER_KEY"], keep="first")
        df_dropped = df_dropped.drop(columns=["CAMPAIGN_LOST"])
    
    # Choose activities to remove and filter out
    #bad_Activities = ["email_send", "email_open", "email_sub", "email_incoming_from_customer", "email_outgoing_to_customer", 
    #                  "initial_creation", "forward_receipt", "phonecall_incoming_from_customer", "phonecall_outgoing_to_customer",
    #                 "text_incoming_from_customer", "text_outgoing_to_customer"]
    #df_filtered = df_dropped[~df_dropped['ACTIVITY_CODE'].isin(bad_Activities)]
    
    # Create dummy variables for ACTIVITY_CODE into 1's and 0's
    df_dummy_code = pd.get_dummies(df_dropped, columns=['ACTIVITY_CODE'], drop_first=True)
    df_dummy_code = df_dummy_code.astype(int)
    
    # Group by CUSTOMER_KEY and sum unique activity codes
    df_grouped = df_dummy_code.groupby('CUSTOMER_KEY').sum()

    # Divide the summed values by the Row_Count for each customer to standardize
    row_counts = df_dummy_code.groupby('CUSTOMER_KEY').size()
    df_grouped['Totals'] = row_counts

    # Add Campaign lost if was stored
    if lost == True:
        df_grouped = df_grouped.merge(lost_column, on="CUSTOMER_KEY", how="left")

    # relable rows to make an ID number
    df_grouped['ID'] = range(len(df_grouped))

    
    return df_grouped

In [30]:
clean_stnl_with_sth = transform_data_AC(stnl_with_sth)
clean_stnl_without_sth = transform_data_AC(stnl_without_sth)

clean_bp_with_sth = transform_data_AC(bp_with_sth)
clean_bp_without_sth = transform_data_AC(bp_without_sth)

clean_sth_with_sth = transform_data_AC(sth_with_sth)
clean_sth_without_sth = transform_data_AC(sth_without_sth)

clean_lost1_with_sth = transform_data_AC(lost1_with_sth)
clean_lost1_without_sth = transform_data_AC(lost1_without_sth)

clean_lost2_with_sth = transform_data_AC(lost2_with_sth)
clean_lost2_without_sth = transform_data_AC(lost2_without_sth)

clean_lost3_with_sth = transform_data_AC(lost3_with_sth)
clean_lost3_without_sth = transform_data_AC(lost3_without_sth)

clean_lost4_with_sth = transform_data_AC(lost4_with_sth)
clean_lost4_without_sth = transform_data_AC(lost4_without_sth)

clean_lost5_with_sth = transform_data_AC(lost5_with_sth)
clean_lost5_without_sth = transform_data_AC(lost5_without_sth)

### Function for standardizing
- Find percentages by dividing by the total number of activities per person 
- Rename variables

In [12]:
def standardize_Activity_Codes(df):
    df_copy = df.copy() 
    activity_cols = df_copy.filter(like="ACTIVITY_CODE").columns

    # Standardize by dividing by 'Totals' 
    #df_copy[activity_cols] = df_copy[activity_cols].div(df_copy['Totals'], axis=0)

    # Rename columns to remove 'ACTIVITY_CODE_'
    df_copy.rename(columns={col: col.replace("ACTIVITY_CODE_", "") for col in activity_cols}, inplace=True)

    return df_copy

### Function for choosing important activity codes(purchases)

In [42]:
def purchases_clean(df):
    columns_to_select = ["primary_purchase_club", "primary_purchase_ga",
                                   "primary_purchase_single","primary_purchase_sth",
                                   "primary_purchase_suite", "secondary_purchase",
                                   "secondary_sale", "merch_purchase", "percent_open", "percent_click", 
                                   "CAMPAIGN_LOST", "Totals"]
    existing_columns = [col for col in columns_to_select if col in df.columns]
    df_purchases = df[existing_columns]
 
    return df_purchases

### Create new variables and use functions

In [43]:
# stnl - w/
clean_stnl_with_sth["percent_open"] = clean_stnl_with_sth["ACTIVITY_CODE_email_open"]/clean_stnl_with_sth["ACTIVITY_CODE_email_send"]
clean_stnl_with_sth["percent_click"] = clean_stnl_with_sth["ACTIVITY_CODE_email_click"]/clean_stnl_with_sth["ACTIVITY_CODE_email_open"]
stnl_with_sth_standard = standardize_Activity_Codes(clean_stnl_with_sth)
cleaner_stnl_with_sth = purchases_clean(stnl_with_sth_standard)
# stnl - w/o
clean_stnl_without_sth["percent_open"] = clean_stnl_without_sth["ACTIVITY_CODE_email_open"]/clean_stnl_without_sth["ACTIVITY_CODE_email_send"]
clean_stnl_without_sth["percent_click"] = clean_stnl_without_sth["ACTIVITY_CODE_email_click"]/clean_stnl_without_sth["ACTIVITY_CODE_email_open"]
stnl_without_sth_standard = standardize_Activity_Codes(clean_stnl_without_sth)
cleaner_stnl_without_sth = purchases_clean(stnl_without_sth_standard)

# bp - w/
clean_bp_with_sth["percent_open"] = clean_bp_with_sth["ACTIVITY_CODE_email_open"]/clean_bp_with_sth["ACTIVITY_CODE_email_send"]
clean_bp_with_sth["percent_click"] = clean_bp_with_sth["ACTIVITY_CODE_email_click"]/clean_bp_with_sth["ACTIVITY_CODE_email_open"]
bp_with_sth_standard = standardize_Activity_Codes(clean_bp_with_sth)
cleaner_bp_with_sth = purchases_clean(bp_with_sth_standard)
# bp - w/o
clean_bp_without_sth["percent_open"] = clean_bp_without_sth["ACTIVITY_CODE_email_open"]/clean_bp_without_sth["ACTIVITY_CODE_email_send"]
clean_bp_without_sth["percent_click"] = clean_bp_without_sth["ACTIVITY_CODE_email_click"]/clean_bp_without_sth["ACTIVITY_CODE_email_open"]
bp_without_sth_standard = standardize_Activity_Codes(clean_bp_without_sth)
cleaner_bp_without_sth = purchases_clean(bp_without_sth_standard)

# sth - w/
clean_sth_with_sth["percent_open"] = clean_sth_with_sth["ACTIVITY_CODE_email_open"]/clean_sth_with_sth["ACTIVITY_CODE_email_send"]
clean_sth_with_sth["percent_click"] = clean_sth_with_sth["ACTIVITY_CODE_email_click"]/clean_sth_with_sth["ACTIVITY_CODE_email_open"]
sth_with_sth_standard = standardize_Activity_Codes(clean_sth_with_sth)
cleaner_sth_with_sth = purchases_clean(sth_with_sth_standard)
# sth - w/o
clean_sth_without_sth["percent_open"] = clean_sth_without_sth["ACTIVITY_CODE_email_open"]/clean_sth_without_sth["ACTIVITY_CODE_email_send"]
clean_sth_without_sth["percent_click"] = clean_sth_without_sth["ACTIVITY_CODE_email_click"]/clean_sth_without_sth["ACTIVITY_CODE_email_open"]
sth_without_sth_standard = standardize_Activity_Codes(clean_sth_without_sth)
cleaner_sth_without_sth = purchases_clean(sth_without_sth_standard)

# lost1 - w/
clean_lost1_with_sth["percent_open"] = clean_lost1_with_sth["ACTIVITY_CODE_email_open"]/clean_lost1_with_sth["ACTIVITY_CODE_email_send"]
clean_lost1_with_sth["percent_click"] = clean_lost1_with_sth["ACTIVITY_CODE_email_click"]/clean_lost1_with_sth["ACTIVITY_CODE_email_open"]
lost1_with_sth_standard = standardize_Activity_Codes(clean_lost1_with_sth)
cleaner_lost1_with_sth = purchases_clean(lost1_with_sth_standard)
# lost1 - w/o
clean_lost1_without_sth["percent_open"] = clean_lost1_without_sth["ACTIVITY_CODE_email_open"]/clean_lost1_without_sth["ACTIVITY_CODE_email_send"]
clean_lost1_without_sth["percent_click"] = clean_lost1_without_sth["ACTIVITY_CODE_email_click"]/clean_lost1_without_sth["ACTIVITY_CODE_email_open"]
lost1_without_sth_standard = standardize_Activity_Codes(clean_lost1_without_sth)
cleaner_lost1_without_sth = purchases_clean(lost1_without_sth_standard)

# lost2 - w/
clean_lost2_with_sth["percent_open"] = clean_lost2_with_sth["ACTIVITY_CODE_email_open"]/clean_lost2_with_sth["ACTIVITY_CODE_email_send"]
clean_lost2_with_sth["percent_click"] = clean_lost2_with_sth["ACTIVITY_CODE_email_click"]/clean_lost2_with_sth["ACTIVITY_CODE_email_open"]
lost2_with_sth_standard = standardize_Activity_Codes(clean_lost2_with_sth)
cleaner_lost2_with_sth = purchases_clean(lost2_with_sth_standard)
# lost2 - w/o
clean_lost2_without_sth["percent_open"] = clean_lost2_without_sth["ACTIVITY_CODE_email_open"]/clean_lost2_without_sth["ACTIVITY_CODE_email_send"]
clean_lost2_without_sth["percent_click"] = clean_lost2_without_sth["ACTIVITY_CODE_email_click"]/clean_lost2_without_sth["ACTIVITY_CODE_email_open"]
lost2_without_sth_standard = standardize_Activity_Codes(clean_lost2_without_sth)
cleaner_lost2_without_sth = purchases_clean(lost2_without_sth_standard)

# lost3 - w/
clean_lost3_with_sth["percent_open"] = clean_lost3_with_sth["ACTIVITY_CODE_email_open"]/clean_lost3_with_sth["ACTIVITY_CODE_email_send"]
clean_lost3_with_sth["percent_click"] = clean_lost3_with_sth["ACTIVITY_CODE_email_click"]/clean_lost3_with_sth["ACTIVITY_CODE_email_open"]
lost3_with_sth_standard = standardize_Activity_Codes(clean_lost3_with_sth)
cleaner_lost3_with_sth = purchases_clean(lost3_with_sth_standard)
# lost3 - w/o
clean_lost3_without_sth["percent_open"] = clean_lost3_without_sth["ACTIVITY_CODE_email_open"]/clean_lost3_without_sth["ACTIVITY_CODE_email_send"]
clean_lost3_without_sth["percent_click"] = clean_lost3_without_sth["ACTIVITY_CODE_email_click"]/clean_lost3_without_sth["ACTIVITY_CODE_email_open"]
lost3_without_sth_standard = standardize_Activity_Codes(clean_lost3_without_sth)
cleaner_lost3_without_sth = purchases_clean(lost3_without_sth_standard)

# lost4 - w/
clean_lost4_with_sth["percent_open"] = clean_lost4_with_sth["ACTIVITY_CODE_email_open"]/clean_lost4_with_sth["ACTIVITY_CODE_email_send"]
clean_lost4_with_sth["percent_click"] = clean_lost4_with_sth["ACTIVITY_CODE_email_click"]/clean_lost4_with_sth["ACTIVITY_CODE_email_open"]
lost4_with_sth_standard = standardize_Activity_Codes(clean_lost4_with_sth)
cleaner_lost4_with_sth = purchases_clean(lost4_with_sth_standard)
# lost4 - w/o
clean_lost4_without_sth["percent_open"] = clean_lost4_without_sth["ACTIVITY_CODE_email_open"]/clean_lost4_without_sth["ACTIVITY_CODE_email_send"]
clean_lost4_without_sth["percent_click"] = clean_lost4_without_sth["ACTIVITY_CODE_email_click"]/clean_lost4_without_sth["ACTIVITY_CODE_email_open"]
lost4_without_sth_standard = standardize_Activity_Codes(clean_lost4_without_sth)
cleaner_lost4_without_sth = purchases_clean(lost4_without_sth_standard)

# lost5 - w/
clean_lost5_with_sth["percent_open"] = clean_lost5_with_sth["ACTIVITY_CODE_email_open"]/clean_lost5_with_sth["ACTIVITY_CODE_email_send"]
clean_lost5_with_sth["percent_click"] = clean_lost5_with_sth["ACTIVITY_CODE_email_click"]/clean_lost5_with_sth["ACTIVITY_CODE_email_open"]
lost5_with_sth_standard = standardize_Activity_Codes(clean_lost5_with_sth)
cleaner_lost5_with_sth = purchases_clean(lost5_with_sth_standard)
# lost5 - w/o
clean_lost5_without_sth["percent_open"] = clean_lost5_without_sth["ACTIVITY_CODE_email_open"]/clean_lost5_without_sth["ACTIVITY_CODE_email_send"]
clean_lost5_without_sth["percent_click"] = clean_lost5_without_sth["ACTIVITY_CODE_email_click"]/clean_lost5_without_sth["ACTIVITY_CODE_email_open"]
lost5_without_sth_standard = standardize_Activity_Codes(clean_lost5_without_sth)
cleaner_lost5_without_sth = purchases_clean(lost5_without_sth_standard)

In [44]:
# stnl
cleaner_stnl_with_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesSTH/cleaner_stnl_with_sth.csv", index=False)
cleaner_stnl_without_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesNoSTH/cleaner_stnl_without_sth.csv", index=False)

# bp
cleaner_bp_with_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesSTH/cleaner_bp_with_sth.csv", index=False)
cleaner_bp_without_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesNoSTH/cleaner_bp_without_sth.csv", index=False)

# sth
cleaner_sth_with_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesSTH/cleaner_sth_with_sth.csv", index=False)
cleaner_sth_without_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesNoSTH/cleaner_sth_without_sth.csv", index=False)

# lost1
cleaner_lost1_with_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesSTH/cleaner_lost1_with_sth.csv", index=False)
cleaner_lost1_without_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesNoSTH/cleaner_lost1_without_sth.csv", index=False)

# lost2
cleaner_lost2_with_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesSTH/cleaner_lost2_with_sth.csv", index=False)
cleaner_lost2_without_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesNoSTH/cleaner_lost2_without_sth.csv", index=False)

# lost3
cleaner_lost3_with_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesSTH/cleaner_lost3_with_sth.csv", index=False)
cleaner_lost3_without_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesNoSTH/cleaner_lost3_without_sth.csv", index=False)

# lost4
cleaner_lost4_with_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesSTH/cleaner_lost4_with_sth.csv", index=False)
cleaner_lost4_without_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesNoSTH/cleaner_lost4_without_sth.csv", index=False)

# lost5
cleaner_lost5_with_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesSTH/cleaner_lost5_with_sth.csv", index=False)
cleaner_lost5_without_sth.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanPurchasesNoSTH/cleaner_lost5_without_sth.csv", index=False)


### With all variables

In [46]:
# stnl
stnl_with_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithSTH/stnl_with_sth_standard.csv", index=False)
stnl_without_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithoutSTH/all_variables_stnl_without_sth.csv", index=False)

# bp
bp_with_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithSTH/bp_with_sth_standard.csv", index=False)
bp_without_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithoutSTH/all_variables_bp_without_sth.csv", index=False)

# sth
sth_with_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithSTH/sth_with_sth_standard.csv", index=False)
sth_without_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithoutSTH/all_variables_sth_without_sth.csv", index=False)

# lost1
lost1_with_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithSTH/lost1_with_sth_standard.csv", index=False)
lost1_without_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithoutSTH/all_variables_lost1_without_sth.csv", index=False)

# lost2
lost2_with_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithSTH/lost2_with_sth_standard.csv", index=False)
lost2_without_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithoutSTH/all_variables_lost2_without_sth.csv", index=False)

# lost3
lost3_with_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithSTH/lost3_with_sth_standard.csv", index=False)
lost3_without_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithoutSTH/all_variables_lost3_without_sth.csv", index=False)

# lost4
lost4_with_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithSTH/lost4_with_sth_standard.csv", index=False)
lost4_without_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithoutSTH/all_variables_lost4_without_sth.csv", index=False)

# lost5
lost5_with_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithSTH/lost5_with_sth_standard.csv", index=False)
lost5_without_sth_standard.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanWithoutSTH/all_variables_lost5_without_sth.csv", index=False)
